# Dead Run RHESSys Workflow with supplied GIS data preparation
<br />
RHESSysWorkflows provides a series of Python tools for performing [RHESSys](https://github.com/RHESSys/RHESSys) data preparation workflows. These tools build on the workflow system defined by [EcohydroLib](https://github.com/selimnairb/EcohydroLib) and [RHESSysWorkflows](https://github.com/selimnairb/RHESSysWorkflows).
<br />


### This notebook assumes general steps 1 to 13 are an existing resource within HydroShare. 

This notebook focuses on general steps 14 to 19 using the Dead Run catchment.<br />
14 Generate template <br />
15 Create world <br />
16 Create flow table <br />
17 Initializing vegetation carbon and nitrogen stores <br />
18 Creating a RHESSys TEC file <br />
19 Running RHESSys models <br />


Users interested in seeing step outputs, remove **output = ** from the command line.

In [ ]:
import os
import logging
from utilities import hydroshare
import shutil
from utilities.rhessys import *
%matplotlib inline

This steps prepares a RHESSys workflow directory

In [ ]:
w = RHESSysWorkflow(project_name='dead_run_Step14', 
                    gageid='custom',
                    start_date='2008-01-01',
                    end_date='2010-01-01'
                    )

Here, your credentials are required to access HydroShare resources.

In [ ]:
hs = hydroshare.hydroshare()

### Please note: This resource is large and retrieving may take a few minutes.

In [ ]:
resource_id = '6dbb0dfb8f3a498881e4de428cb1587c'
# get ETV data bundle for this tutotial stored on HydroShare
content = hs.getResourceFromHydroShare(resource_id)

### Please note: This resource is large and unzipping may take a few minutes.
The above step prints out the zip contents filename. For Deadrun example, the filename is 'DR5_3m_nonburned_DEM_rain_duration_DEM_float_lctest_raingarden.zip'. <br>
If you are using a different resource, modify the zipfilepathname = hs.content[ **YOUR FILE NAME HERE** ] step below.

In [ ]:
zipfolder = w.sub_project_folder + '/RHESSys_ETV'
w.create_path(zipfolder)
zipfilepathname = hs.content['DR5_3m_nonburned_DEM_rain_duration_DEM_float_lctest_raingarden.zip']
w.unzip_etv_zip_file_at_path(zipfilepathname, zipfolder)
print 'Unzip Finished'

RHESSys workflows requires GDAL, this step defines the GDAL_DATA environment path.

In [ ]:
os.environ["GDAL_DATA"] = "/opt/conda/share/gdal"

## Download RHESSys Model software from HydroShare resource

The Deadrun resource contains a precompiled RHESSys model and tools. However, the code has been compiled on a OSX environment different OS from the Jupyter notebooks. Hence, the RHESSys model will not execute. <br />
<br />
As a notebook user, you have two options. <br />
 <br />
**Option #1:** As demonstrated with other RHESSys notebooks, a step called `ImportRHESSysSource` will import the latest RHESSys source code and compile the code on the host. This method is recommended if you desire the latest model and the data preparation steps (1 to 14) are compatiable.
 <br />
**Option #2:** This notebook, demonstrates using an existing compiled RHESSys model and tools, kept on HydroShare as a resource called **'rhessys5.18.r2'**.
<br>



In [ ]:
resource_id = 'f92ccd46170b4d4cac028b7765536d75' ##Resource id for rhessys5.18.r2
# get ETV data bundle for this tutotial stored on HydroShare
content = hs.getResourceFromHydroShare(resource_id)

In [ ]:
zipfolder = w.sub_project_folder + '/RHESSys_ETV/DR5_3m_nonburned_DEM_rain_duration_DEM_float_lctest_raingarden/rhessys/bin'
zipfilepathname = hs.content['rhessys5.18.r2.zip']
w.unzip_etv_zip_file_at_path(zipfilepathname, zipfolder)
print 'Unzip Finished'

The permissions of the file contents need to be modified for execution.

In [ ]:
print(zipfolder)
os.chmod(zipfolder + '/cf10.0b3', 0777)
os.chmod(zipfolder + '/cf10.0b2', 0777)
os.chmod(zipfolder + '/g2w', 0777)
os.chmod(zipfolder + '/lairead', 0777)
os.chmod(zipfolder + '/rat', 0777)
os.chmod(zipfolder + '/rhessys5.18.r2', 0777)
print 'Permissions Finished'

Here, the sub project folder is modified.

In [ ]:
w.sub_project_folder += "/RHESSys_ETV/DR5_3m_nonburned_DEM_rain_duration_DEM_float_lctest_raingarden"

## Step 14 Generate World Template

Now we are almost ready to create the world file for our watershed. First we must create the template from which the world file will be created. To do this, we'll use the **GenerateWorldTemplate** tool. Fortunately this is very easy because the metadata store contains nearly all the information needed to create the template.

The -c (a.k.a. --climateStation) option specifies the climate station to associate with this world file template. The --aspectMinSlopeOne option is necessary work around limitations in the program used to create the world file when the input DEM has areas of low slope.

### Note: If you used a different resource from HydroShare, you will need to modify the climate station parameter.

In [ ]:
world_options = " -p " + w.sub_project_folder + " -c dr5_composite5"
print(world_options)
w.GenerateWorldTemplateOptions(world_options)
print 'GenerateWorldTemplate Finished'

## Step 15 Create World File

Now use the **CreateWorldfile** tool to create a world file using this template.
Note: Output from CreateWorldfile has been piped to output variable. For debugging, uncomment print statement (i.e. remove #).

In [ ]:
print(w.sub_project_folder)
output = w.CreateWorldfile(w.sub_project_folder)
#print(output) ##Note Output message VERY Long
print 'CreateWorldfile Finished'

## Step 16 Create Flow Table
As with worldfile creation, at this point in the workflow, RHESSysWorkflows's metadata store contains all the information needed to create a flow table using the createflowpaths (CF) RHESSys program. We'll use CreateFlowtable to create our flow table.

This will result in the creation of a flow table called world.flow in the flow directory of your rhessys directory. Now we have almost everything we need to run RHESSys simulations.

See the RHESSysWorkflows tutorial to learn how to [route surface flows for road pixels directly to the stream](https://github.com/selimnairb/RHESSysWorkflows#creating-the-flow-table) and [create surface flow tables using a roof connectivity map](https://github.com/selimnairb/RHESSysWorkflows#creating-a-surface-flow-table-using-a-roof-connectivity-map)


In [ ]:
#flow_options = " -p " + w.sub_project_folder + " --routeRoads --routeRoofs -v"
flow_options = " -p " + w.sub_project_folder
print(flow_options)
output = w.CreateFlowtableOptions(flow_options)
print(output)
print('CreateFlowtableOptions Finished')

## Step 17 Initializing vegetation carbon and nitrogen stores
RHESSys provides a program called LAIread to initialize vegetation carbon and nitrogen stores in your world file.

    Note, LAIread should only be used for RHESSys simulations with static vegetation (i.e. not dynamic vegetation mode enable via the -g command line option to RHESSys).

Initializing carbon and nitrogen stores is a multi-step process that involves running LAI read to generate a redefine worldfile, running a 3-day RHESSys simulation to incorporate the redefine worldfile, writing out a new worldfile with initialized vegetation carbon and nitrogen stores. RHESSysWorkflows automates all of these processes for you using the tool RunLAIRead, which can even figure out what date to start the 3-day RHESSys simulation on based on your climate data.

    In the current version of RHESSysWorkflows, RunLAIRead is only able to read simulation start dates from point time-series climate data. Users of ASCII or NetCDF gridded climate data must run LAI read by hand. The next release of RHESSysWorkflows will add support for gridded climate data to RunLAIRead.

To initialize vegetation carbon and nitrogen stores, LAIread relies on allometric relationships between leaf area and carbon and nitrogen mass in various plant tissues (e.g. leaf, stem, root). Consult the RHESSys wiki for more information on allometric relationships used by LAIread. These allometric parameters have not yet been added to the RHESSys ParamDB database proper. A default version of the parameters for RHESSys base vegetation classes is stored in the RHESSys ParamDB source coderepository. RHESSysWorkflows stores this file under the name allometric.txt in the allometry folder of the ParamDB of your rhessys/db folder. 


In [ ]:
output = w.RunLAIRead(w.sub_project_folder)
#print(output) ##Note Output message VERY Long
print 'RunLAIRead Finished'

## Step 18 Creating a RHESSys TEC file
We need one more thing before we can run our model, a TEC file. TEC stands for "temporal event controller". We use a TEC file to tell RHESSys to do things on at certain times during a simulation.

#### With this notebook, the supplied **tec_daily.txt** will be used. If you desire a short test, or different times, uncomment the code below.

In [ ]:
###w.RunCmd(w.sub_project_folder, 3)

## Step 19 Running RHESSys models
Once you have built a RHESSys model using RHESSysWorkflows, you are free to run your model manually. However this will not capture information about model runs in your project metadata. If you would like to record your runs in your project metadata, use the **RunModel** command.


Because the project metadata knows where RHESSys is installed in your project directory, you don't have to specify the full path of any of the RHESSys input files (e.g. world files, flow tables, TEC files, etc), you only need to specify the filenames. RunModel will echo RHESSys console outlet to the screen (if the -v or verbose option is specified as above), and will always save the same output into a file named 'rhessys.out' stored in the output folder for each model run. The output folder will be named based on the value you provide for the '-pre' or output prefix option. Model output is stored in the output directory of the rhessys directory of your project directory.

For more details on using RunModel see the RHESSysWorkflows [tutorial](https://github.com/selimnairb/RHESSysWorkflows#running-rhessys-models).

In [ ]:
#options = " -p " + w.sub_project_folder + " -c dr5_composite3"
runmodel_options = " -p " + w.sub_project_folder + ' -d \"Test model run\" --basin -pre test -st 2008 1 1 1 -ed 2010 1 1 1 -w world_init -t tec_daily.txt -r world.flow -- -s 0.07041256017 133.552915269 1.81282283058 -sv 4.12459677088 78.3440566535 -gw 0.00736592779294 0.340346799457'

w.RunModelOptions(runmodel_options)